<a href="http://www.calstatela.edu/centers/hipic"><img align="left" src="https://avatars2.githubusercontent.com/u/4156894?v=3&s=100"><image/>
</a>
<img align="right" alt="California State University, Los Angeles" src="http://www.calstatela.edu/sites/default/files/groups/California%20State%20University%2C%20Los%20Angeles/master_logo_full_color_horizontal_centered.svg" style="width: 360px;"/>

#    CIS5560 Term Project Tutorial

------
#### Authors: [Monika Mishra](https://www.linkedin.com/in/monika-mishra-8b2a4115/), [Amogh Mahesh](https://www.linkedin.com/in/amoghmahesh/), [Aakanksha Tasgaonkar](https://www.linkedin.com/in/aakanksha-tasgaonkar-272ba393/)

#### Instructor: [Jongwook Woo](https://www.linkedin.com/in/jongwook-woo-7081a85)

#### Date: 05/16/2019

## Import Spark SQL and Spark ML Libraries
Import all the Spark SQL and ML libraries as mentioned below. This is neccessary to access the functions available in those libraries.

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import LogisticRegression

##Read csv file from DBFS  (Databricks File System)
1. After <filename>.csv file is added to the data in the left frame, create a table using the UI, "Upload File"
2. Click "Preview Table to view the table" and select the option as <filename>.csv has a header as the first row. "First line is header"
3. Change the data type of the columns
4. Click on the create table button.

In [6]:
# DataFrame Schema, that should be a Table schema
amazon1 = StructType([
  StructField("marketplace", StringType(), False),
  StructField("customer_id", IntegerType(), False),
  StructField("review_id", IntegerType(), False),
  StructField("product_id", StringType(), False),
  StructField("product_parent", IntegerType(), False),
  StructField("product_title", StringType(), False),
  StructField("product_category", StringType(), False),
  StructField("star_rating", DoubleType(), False),
  StructField("helpful_votes", IntegerType(), False),
  StructField("total_votes", IntegerType(), False),
  StructField("vine", StringType(), False),
  StructField("verified_purchase", StringType(), False),
  StructField("review_headline", StringType(), False),
  StructField("review_body", StringType(), False),
  StructField("review_date", StringType(), False)
])

##Create a dataframe from the table, using Spark SQL

In [8]:
tsv = spark.sql("SELECT * FROM ratings_csv")

tsv.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+
marketplace|customer_id| review_id|product_id|product_parent| product_title| product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+
 US| 16334773|R13IVTRU56Q4VM|B00RNRD8RA| 674964264| The Interview|Digital_Video_Dow...| 4.0| 0| 1| N| Y| Four Stars| Hilarious!|3/20/15 0:00|
 US| 12167164| RANZMY3F0R8CO|B000002IM7| 199692198|Out Of The Silent...| Music| 5.0| 3| 4| N| N|The emergence of ...|This was the debu...|5/10/07 0:00|
 US| 9946096|R1MK8B4I4333CN|B00770KR7W| 227474890|McFarlane Toys Ha...| Toys| 4.0| 0| 0| N| Y|The armor accesso...|The armor accesso...|8/28/14 0:00|
 US| 52224424|R2GW34QCBXAROM|B005GHN2GG| 821592226| Master of Crows|Digital_Ebook_Pur...| 2.0| 0| 1| N| Y|Not Fantasy: it's...|Well-written, alt...| 4/8/15 0:00|
 US| 51712853|R3S0HJBNS0CCV6|B004EPZ020| 881962457|Paranormal Activi...| Video DVD| 3.0| 4| 8| N| N|Paranormal Activi...|Much like before,...|11/4/11 0:00|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+
only showing top 5 rows

## Clear the data

In the next step, we will select the columns that are useful for our model.

In [10]:
tsv1 = tsv.select("product_id", "product_parent", "product_title", "product_category", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body", "review_date","star_rating" )

tsv1.show(5)

+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+-----------+
product_id|product_parent| product_title| product_category|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|star_rating|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+-----------+
B00RNRD8RA| 674964264| The Interview|Digital_Video_Dow...| 0| 1| N| Y| Four Stars| Hilarious!|3/20/15 0:00| 4.0|
B000002IM7| 199692198|Out Of The Silent...| Music| 3| 4| N| N|The emergence of ...|This was the debu...|5/10/07 0:00| 5.0|
B00770KR7W| 227474890|McFarlane Toys Ha...| Toys| 0| 0| N| Y|The armor accesso...|The armor accesso...|8/28/14 0:00| 4.0|
B005GHN2GG| 821592226| Master of Crows|Digital_Ebook_Pur...| 0| 1| N| Y|Not Fantasy: it's...|Well-written, alt...| 4/8/15 0:00| 2.0|
B004EPZ020| 881962457|Paranormal Activi...| Video DVD| 4| 8| N| N|Paranormal Activi...|Much like before,...|11/4/11 0:00| 3.0|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+-----------+
only showing top 5 rows

## Clean the data

In [12]:
df1 = tsv1.filter(tsv1.product_category.isNotNull())
df2 = df1.filter(df1.helpful_votes.isNotNull())
df3 = df2.filter(df2.total_votes.isNotNull())
df4 = df3.filter(df3.vine.isNotNull())
df5 = df4.filter(df4.verified_purchase.isNotNull())
df6 = df5.filter(df5.review_headline.isNotNull())
df7 = df6.filter(df6.review_body.isNotNull())
df8 = df7.filter(df7.review_date.isNotNull())
df9 = df8.filter(df8.star_rating.isNotNull())

df9.show(5)

+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+-----------+
product_id|product_parent| product_title| product_category|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|star_rating|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+-----------+
B00RNRD8RA| 674964264| The Interview|Digital_Video_Dow...| 0| 1| N| Y| Four Stars| Hilarious!|3/20/15 0:00| 4.0|
B000002IM7| 199692198|Out Of The Silent...| Music| 3| 4| N| N|The emergence of ...|This was the debu...|5/10/07 0:00| 5.0|
B00770KR7W| 227474890|McFarlane Toys Ha...| Toys| 0| 0| N| Y|The armor accesso...|The armor accesso...|8/28/14 0:00| 4.0|
B005GHN2GG| 821592226| Master of Crows|Digital_Ebook_Pur...| 0| 1| N| Y|Not Fantasy: it's...|Well-written, alt...| 4/8/15 0:00| 2.0|
B004EPZ020| 881962457|Paranormal Activi...| Video DVD| 4| 8| N| N|Paranormal Activi...|Much like before,...|11/4/11 0:00| 3.0|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+--------------------+--------------------+------------+-----------+
only showing top 5 rows

##Select features and label
1. Select the relevant columns in a new dataframe and define the label.
2. Convert the string type columns into indexes using StringIndexer
3. Split the data in 70-30 train-test ratio.
4. Store the train-test data for both of the models separately.
5. Using Vector Assembler define the set of columns to be used as features.

In [14]:
data = df9.select("product_id", "product_parent", "product_title", "product_category", "helpful_votes", "total_votes", "vine", "verified_purchase",  "review_date",col("star_rating").alias("label"))

data.show(5)

+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+------------+-----+
product_id|product_parent| product_title| product_category|helpful_votes|total_votes|vine|verified_purchase| review_date|label|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+------------+-----+
B00RNRD8RA| 674964264| The Interview|Digital_Video_Dow...| 0| 1| N| Y|3/20/15 0:00| 4.0|
B000002IM7| 199692198|Out Of The Silent...| Music| 3| 4| N| N|5/10/07 0:00| 5.0|
B00770KR7W| 227474890|McFarlane Toys Ha...| Toys| 0| 0| N| Y|8/28/14 0:00| 4.0|
B005GHN2GG| 821592226| Master of Crows|Digital_Ebook_Pur...| 0| 1| N| Y| 4/8/15 0:00| 2.0|
B004EPZ020| 881962457|Paranormal Activi...| Video DVD| 4| 8| N| N|11/4/11 0:00| 3.0|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+------------+-----+
only showing top 5 rows

In [15]:
data = StringIndexer(inputCol='product_id', outputCol='product_id'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='product_title', outputCol='product_title'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='product_category', outputCol='product_category'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='vine', outputCol='vine'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='verified_purchase', outputCol='verified_purchase'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='review_date', outputCol='review_date'+"_index").fit(data).transform(data)

data.show(5)

+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+------------+-----+----------------+-------------------+----------------------+----------+-----------------------+-----------------+
product_id|product_parent| product_title| product_category|helpful_votes|total_votes|vine|verified_purchase| review_date|label|product_id_index|product_title_index|product_category_index|vine_index|verified_purchase_index|review_date_index|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+------------+-----+----------------+-------------------+----------------------+----------+-----------------------+-----------------+
B00RNRD8RA| 674964264| The Interview|Digital_Video_Dow...| 0| 1| N| Y|3/20/15 0:00| 4.0| 419.0| 332.0| 3.0| 0.0| 0.0| 174.0|
B000002IM7| 199692198|Out Of The Silent...| Music| 3| 4| N| N|5/10/07 0:00| 5.0| 8778.0| 8382.0| 5.0| 0.0| 1.0| 3453.0|
B00770KR7W| 227474890|McFarlane Toys Ha...| Toys| 0| 0| N| Y|8/28/14 0:00| 4.0| 5380.0| 4937.0| 7.0| 0.0| 0.0| 472.0|
B005GHN2GG| 821592226| Master of Crows|Digital_Ebook_Pur...| 0| 1| N| Y| 4/8/15 0:00| 2.0| 3234.0| 3361.0| 1.0| 0.0| 0.0| 296.0|
B004EPZ020| 881962457|Paranormal Activi...| Video DVD| 4| 8| N| N|11/4/11 0:00| 3.0| 24673.0| 5950.0| 2.0| 0.0| 1.0| 1529.0|
+----------+--------------+--------------------+--------------------+-------------+-----------+----+-----------------+------------+-----+----------------+-------------------+----------------------+----------+-----------------------+-----------------+
only showing top 5 rows

## Split the data
In the next step we split the data in a train and test set. We have split the data in the ratio of **70 to 30**.

In [17]:
# Split the data
splits = data.randomSplit([0.7, 0.3])


# for gradient boosted tree regression
gbt_train = splits[0]
gbt_test = splits[1].withColumnRenamed("label", "trueLabel")

print ("Training Rows:", gbt_train.count(), " Testing Rows:", gbt_test.count())

gbt_train.show(5)

Training Rows: 97051 Testing Rows: 41546
+----------+--------------+--------------------+----------------+-------------+-----------+----+-----------------+-------------+-----+----------------+-------------------+----------------------+----------+-----------------------+-----------------+
product_id|product_parent| product_title|product_category|helpful_votes|total_votes|vine|verified_purchase| review_date|label|product_id_index|product_title_index|product_category_index|vine_index|verified_purchase_index|review_date_index|
+----------+--------------+--------------------+----------------+-------------+-----------+----+-----------------+-------------+-----+----------------+-------------------+----------------------+----------+-----------------------+-----------------+
006072448X| 467980841|We Need to Talk A...| Books| 145| 168| N| N| 3/18/05 0:00| 3.0| 7243.0| 3101.0| 4.0| 0.0| 1.0| 4214.0|
006085880X| 990695172|The Pact: A Love ...| Books| 0| 1| N| N| 8/4/14 0:00| 5.0| 7788.0| 442.0| 4.0| 0.0| 1.0| 127.0|
006145205X| 277507408|The Rational Opti...| Books| 0| 2| N| Y| 9/19/10 0:00| 5.0| 12102.0| 10880.0| 4.0| 0.0| 0.0| 2305.0|
006202406X| 766861888|Allegiant (Diverg...| Books| 0| 2| N| N| 7/2/15 0:00| 5.0| 313.0| 383.0| 4.0| 0.0| 1.0| 946.0|
006202406X| 766861888|Allegiant (Diverg...| Books| 2| 2| N| N|11/30/13 0:00| 1.0| 313.0| 383.0| 4.0| 0.0| 1.0| 531.0|
+----------+--------------+--------------------+----------------+-------------+-----------+----+-----------------+-------------+-----+----------------+-------------------+----------------------+----------+-----------------------+-----------------+
only showing top 5 rows

In [18]:
assembler = VectorAssembler(inputCols = ["product_parent", "helpful_votes", "total_votes", "product_id_index", "product_title_index", "product_category_index", "vine_index", "verified_purchase_index","review_date_index"], outputCol="features")

## Algorithms used to Train the model

In this project we have used the following algorithm to train our model:

1. GBT Regressor

## 2. Gradient Boosted Tree Regression Model

In [21]:
gbt = GBTRegressor(featuresCol='features', labelCol='label', maxBins=77582,  maxIter=2)
gbt_pipeline = Pipeline(stages=[assembler, gbt])

In [22]:
#paramGrid1 = ParamGridBuilder().addGrid(gbt.maxDepth, [2,3,4,5,6]).build()

paramGrid1 = (ParamGridBuilder()
              .addGrid(gbt.maxDepth,[2,3])
              .addGrid(gbt.minInfoGain,[0.0, 0.1, 0.2, 0.3])
              .addGrid(gbt.stepSize,[0.05, 0.1, 0.2, 0.4])
              .build())

#cv1 = CrossValidator(estimator=gbt_pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid1, numFolds=2)
gbt_tvs = TrainValidationSplit(estimator=gbt_pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid1, trainRatio=0.8)

In [23]:
gbt_model = gbt_tvs.fit(gbt_train)

In [24]:
gbt_prediction = gbt_model.transform(gbt_test)
gbt_predicted = gbt_prediction.select("features", "prediction", "trueLabel")
gbt_predicted.show(10)

+--------------------+------------------+---------+
 features| prediction|trueLabel|
+--------------------+------------------+---------+
[5.87492563E8,14....|3.1771068780786855| 5.0|
[1.65248566E8,11....|3.1771068780786855| 1.0|
[1.83823737E8,2.0...| 4.514860755037212| 3.0|
[2.3266935E8,5.0,...|3.9370171046782154| 3.0|
[7.66861888E8,0.0...|3.5903050723937073| 5.0|
[7.66861888E8,6.0...|3.5903050723937073| 1.0|
[3.24116661E8,0.0...| 4.32316117996384| 5.0|
[3.24116661E8,0.0...| 4.32316117996384| 5.0|
[3.24116661E8,0.0...| 4.32316117996384| 5.0|
[5.82629146E8,0.0...| 4.514860755037212| 5.0|
+--------------------+------------------+---------+
only showing top 10 rows

## Test the model
We transform the test dataframe to generate label predictions.

### Retrieve the Root Mean Square Error (RMSE)
There are a number of metrics used to measure the variance between predicted and actual values. Of these, the root mean square error (RMSE) is a commonly used value that is measured in the same units as the prediced and actual values - so in this case, the RMSE indicates the average number of minutes between predicted and actual flight delay values. You can use the **RegressionEvaluator** class to retrieve the RMSE.

In [27]:
gbt_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
gbt_rmse = gbt_evaluator.evaluate(gbt_prediction)
print ("Root Mean Square Error (RMSE):", gbt_rmse)

Root Mean Square Error (RMSE): 1.1119769286966736

#### Root Mean Square Error (RMSE) for Gradient Boosted Tree Regression Model: 1.1119769286966736